# TESTI: itse määriteltyjä sarakkeita esikäsittelijällä

Tässä testissä testataan osaa itse määritellyistä sarakkeista, jotka myös esikäsittelijä osaa luoda. Lisäksi testataan uutta mode-valikoimaa, joka on nyt `['none', 'minimal', 'common']`.

## Mode: 'common'

'common' on oletus, joten sitä ei välttämättä tarvitse antaa kutsun parametriksi. Mode pyrkii karsimaan virheellisen datan pois ja tekee kaikki pyydetyt itse määritellyt sarakkeet. Tässä testataan:

- 'customer_segment_3'
  - Jako: RES, SMALL NON-RES, LARGE NON-RES
  - NON-RES koostettu alkuperäisistä ryhmistä COM, GOV, SCHOOL, NON-PROFIT, NON-RES
  - Raja määritellään parametrilla `large-limit`, joka on oletuksena 100 kW
  - Raja-arvo kuuluu ryhmään 'SMALL NON-RES'
- 'battery_storage'
  - bool sarake
  - arvo 1 : järjestelmässä (arviolta) on akku
  - arvo 0 : järjestelmässä (arviolta) ei ole akkua
  - arvioon voi sisällyttää teslan asennuksia kolmella tavalla
    - 'none' asettaa kaikkiin arvon 0
    - 'neutral' antaa akun määräytyä muiden kriteerien perusteella
    - 'all' asettaa kaikkiin arvon 1

In [1]:
import importlib.util
import sys
file_path = "../Tehtava-03/preprocessor.py"; module_name = "preprocessor"; spec = importlib.util.spec_from_file_location(module_name, file_path)
esik = importlib.util.module_from_spec(spec); sys.modules[module_name] = esik; spec.loader.exec_module(esik)

cols = ['customer_segment_3', 'system_size_DC', 'battery_storage', 'installer_name']

df_none = esik.esik(cols, large_limit=120, tesla_battery='none')
df_neutral = esik.esik(cols, large_limit=120, tesla_battery='neutral', sshi_with_battery=False)
df_all = esik.esik(cols, large_limit=120, tesla_battery='all')


In [2]:
print("df_all.shape:", df_all.shape)
display(df_all.describe())
display(df_all.groupby(['customer_segment_3']).agg({'system_size_DC': ['size', 'count', 'min', 'max'], 'battery_storage': ['count', 'mean']}))
display(df_all.groupby(['battery_storage']).agg({'system_size_DC': ['size', 'count', 'min', 'max']}))


df_all.shape: (1000000, 4)


,system_size_DC,battery_storage
count,808686.000000,1000000.000000
mean,9.228860,0.180785
std,47.269687,0.384840
min,0.068000,0.000000
25%,3.900000,0.000000
50%,5.440000,0.000000
75%,7.590000,0.000000
max,13305.610000,1.000000


system_size_DC                            battery_storage  \
                             size   count      min       max           count   
customer_segment_3                                                             
LARGE NON-RES                5278    5278  120.060  13305.61            5278   
NON-RES                      5330       0      NaN       NaN            5330   
RES                        974311  788327    0.068   1106.56          974311   
SMALL NON-RES               15081   15081    0.230    120.00           15081   

                              
                        mean  
customer_segment_3            
LARGE NON-RES       0.101933  
NON-RES             0.102064  
RES                 0.183033  
SMALL NON-RES       0.090975

system_size_DC                           
                          size   count    min         max
battery_storage                                          
0                       819215  657604  0.068   7586.5625
1                       180785  151082  0.225  13305.6100

### Akut Tesla Energyllä

In [3]:
df = df_none.copy()
df = df[(df['battery_storage'] == 1) & (df['installer_name'] == 'Tesla Energy')]
print("df_none.shape: ", df.shape)
df = df_neutral.copy()
df = df[(df['battery_storage'] == 1) & (df['installer_name'] == 'Tesla Energy')]
print("df_neutral.shape: ", df.shape)
df = df_all.copy()
df = df[(df['battery_storage'] == 1) & (df['installer_name'] == 'Tesla Energy')]
print("df_all.shape: ", df.shape)
# sshi_battery True
# 0
# 4808
# 170110

df_none.shape:  (0, 4)
df_neutral.shape:  (4774, 4)
df_all.shape:  (170110, 4)


## Mode: 'none'

### Itse määriteltyjä sarakkeita

Kielletään varsinainen esikäsittely, mutta yritetään ladata itse määriteltyjä sarakkeita. Pitäisi tulostaa huomautuksen ja ladata vain alkuperäisiin kuuluvat sarakkeet.

In [4]:
cols = ['customer_segment_3', 'system_size_DC', 'battery_storage']
df_all = esik.esik(cols, 'none')
print("df_all.shape:", df_all.shape)
display(df_all.describe())


############################################################
Itse määriteltyjä sarakkeita ei ladattu.
############################################################
df_all.shape: (1000000, 1)


,system_size_DC
count,988692.000000
mean,9.325515
std,47.065020
min,0.005500
25%,3.924000
50%,5.520000
75%,7.750000
max,13305.610000


### Vain alkuperäisiä sarakkeita

Testataan, että mode toimii myös oikein käytettynä.

In [5]:
cols = ['total_installed_price', 'system_size_DC']
df_all = esik.esik(cols, 'none')
print("df_all.shape:", df_all.shape)
display(df_all.describe())


df_all.shape: (1000000, 2)


,total_installed_price,system_size_DC
count,8.143640e+05,988692.000000
mean,3.824708e+04,9.325515
std,1.919481e+05,47.065020
min,1.000000e-02,0.005500
25%,1.628200e+04,3.924000
50%,2.404256e+04,5.520000
75%,3.424995e+04,7.750000
max,6.150719e+07,13305.610000
